In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
#oof_df_1 = pd.read_csv('../../01_Baseline/exp/result/01_v1_13/oof_df.csv')
oof_df_1 = pd.read_csv('../../01_Baseline/exp/result/01_v1_30/oof_df.csv')
#oof_df_2 = pd.read_csv('../../01_Baseline/exp/result/01_v1_16/oof_df.csv')
oof_df_2 = pd.read_csv('../../04_Aug/exp/result/04_v1_03/oof_df.csv')
oof_df_3 = pd.read_csv('../../01_Baseline/exp/result/01_v1_24/oof_df.csv')
oof_df_4 = pd.read_csv('../../01_Baseline/exp/result/01_v1_25/oof_df.csv')
oof_df_5 = pd.read_csv('../../01_Baseline/exp/result/01_v1_45/oof_df.csv')

oof_df_1 = train_df[['text_id']].merge(oof_df_1, how='left', on='text_id')
oof_df_2 = train_df[['text_id']].merge(oof_df_2, how='left', on='text_id')
oof_df_3 = train_df[['text_id']].merge(oof_df_3, how='left', on='text_id')
oof_df_4 = train_df[['text_id']].merge(oof_df_4, how='left', on='text_id')
oof_df_5 = train_df[['text_id']].merge(oof_df_5, how='left', on='text_id')

num_models = 5


TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = oof_df_1[TARGET_COLS].values
preds += oof_df_2[TARGET_COLS].values
preds += oof_df_3[TARGET_COLS].values
preds += oof_df_4[TARGET_COLS].values
preds += oof_df_5[TARGET_COLS].values
preds /= num_models


oofs = []
oofs.append(oof_df_1[TARGET_COLS].values)
oofs.append(oof_df_2[TARGET_COLS].values)
oofs.append(oof_df_3[TARGET_COLS].values)
oofs.append(oof_df_4[TARGET_COLS].values)
oofs.append(oof_df_5[TARGET_COLS].values)

In [4]:
oof_df = oof_df_1.copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4472


In [7]:
ensemble_predictions=np.stack(oofs)

In [8]:
def ensemble_score(weights,return_pred=False):
    weights=np.array(weights)
    weights=weights.reshape(-1,1,1)/weights.sum()
    p=weights.reshape(-1,1,1)*ensemble_predictions
    p=p.sum(0)
    score=calc_metric(p, train_df[TARGET_COLS])
    if return_pred:
        return score,p
    else:
        return score   

In [9]:
from skopt import gp_minimize

results=gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                    verbose=True,random_state=2022)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0028
Function value obtained: 0.4473
Current minimum: 0.4473
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0018
Function value obtained: 0.4485
Current minimum: 0.4473
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0018
Function value obtained: 0.4480
Current minimum: 0.4473
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0018
Function value obtained: 0.4481
Current minimum: 0.4473
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0018
Function value obtained: 0.4472
Current minimum: 0.4472
Iteration No: 6 started. Evaluating

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.5374
Function value obtained: 0.4475
Current minimum: 0.4466
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.6659
Function value obtained: 0.4466
Current minimum: 0.4466
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.5166
Function value obtained: 0.4471
Current minimum: 0.4466
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.5718
Function value obtained: 0.4475
Current minimum: 0.4466
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.5069
Function value obtained: 0.4466
Current minimum: 0.4466
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.8269
Function value obtained: 0.4466
Current minimum: 0.4466
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.8134
Function value obtained: 0.4480
Current minimum: 0.4466
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.8724
Function value obtained: 0.4495
Current minimum: 0.4466
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.8811
Function value obtained: 0.4476
Current minimum: 0.4466
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.8598
Function value obtained: 0.4466
Current minimum: 0.4466
Iteration No: 86 started. Sea

In [10]:
best_weights=np.array(results['x'])/sum(results['x'])
print(best_weights)

[0.28698886 0.30226666 0.03373453 0.14339509 0.23361486]


In [11]:
# smaller is better for this metric
score,ensemble_pred=ensemble_score(best_weights,True)
score

0.446613764639429

In [12]:
print('CV={:.4f}'.format(score))

CV=0.4466
